In [2]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install modin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.8 MB/s eta 0:00:00


In [8]:
import ssl
from functools import reduce
import ray
import nest_asyncio
import modin.pandas as pd
import requests
from bs4 import BeautifulSoup
ssl._create_default_https_context = ssl._create_unverified_context
url = "https://h1bdata.info/index.php?em=&job=quantitative+&city=&year=2023"
BASELINE = 175000

In [17]:
@ray.remote
def _get_df(title):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    records = []
    for tr in table.findAll("tr"):
        trs = tr.findAll("td")
        record = []
        for each in trs:
            try:
                link = each.find('a')['href']
                text = each.text
                record.append(link)
                record.append(text)
            except:
                text = each.text
                record.append(text)
        records.append(record)

    df = pd.DataFrame(data=records)
    df = df.iloc[1:, :].loc[:, [1,3,4,5,7,8,9]]
    df.columns = ["FIRM", "TITLE", "HREF", "RAW_BASE", "LOCATION", "SUBMIT_DATE", "START_DATE"]
    def _get_lca_wage(x):
        import time
        if x["HREF"] is None:
            return 0
        url = f"https://h1bdata.info/{x['HREF']}"
        for i in range(3):
            try:
                accurate_df = pd.read_html(url)[0]
            except:
                time.sleep(i)
        dic = accurate_df.to_dict(orient='records')
        new_dict = {}
        cand1, cand2 = None, None
        for sub_dict in dic:
            new_dict[sub_dict[0]] = sub_dict[1]
        if "LCA CASE WAGE RATE FROM" in new_dict:
            cand1 = new_dict["LCA CASE WAGE RATE FROM"]
        if "LCA CASE WAGE RATE TO" in new_dict:
            cand2 = new_dict["LCA CASE WAGE RATE TO"]
        if cand1 is None:
            return cand2
        if cand2 is None:
            return cand1
        return max(cand1, cand2)
    df["HREF_BASE"] = df.apply(lambda x: _get_lca_wage(x), axis=1)
    df["BASE"] = df[['HREF_BASE', 'RAW_BASE']].max(axis=1)
    return df

In [18]:
refs = []
titles = ['quantitative+', 'senior+quantitative+', 'quant+']
for title in titles:
    refs.append(_get_df.remote(title))

In [19]:
res = ray.get(refs)

(_get_df pid=19103) UserWarning: Distributing <class 'list'> object. This may take some time.
(_deploy_ray_func pid=19316) UserWarning: Distributing <class 'list'> object. This may take some time.
(_deploy_ray_func pid=19316) UserWarning: `read_html` is not currently supported by PandasOnRay, defaulting to pandas implementation.
(_deploy_ray_func pid=19221) Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.
(_get_df pid=19104) UserWarning: Distributing <class 'list'> object. This may take some time. [repeated 2x across cluster]
(_deploy_ray_func pid=19316) UserWarning: `read_html` is not currently supported by PandasOnRay, defaulting to pandas implementation. [repeated 10x across cluster]
(_deploy_ray_func pid=19316) UserWarning: `read_html` is not currently supported by PandasOnRay, defaulting to pandas implementation. [repeated 11x across cluster]
(_deploy_ray_func pid=19316) UserWarning: `read_html` is not currently suppo

In [20]:
df = reduce(lambda df1, df2: pd.concat([df1, df2], ignore_index=True), res)
df = df[df['BASE'] >= BASELINE] # IDGAF
df = df[df["BASE"]<700_000] # Most certainly data entry error

(_deploy_ray_func pid=19104) FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
(_deploy_ray_func pid=19104) FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


In [21]:
df = df.sort_values(by="BASE", ascending=False)

In [22]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df

In [24]:
df['FIRM'].unique()

array(['VIKING GLOBAL INVESTORS LP', 'D E SHAW & CO LP',
       'CITADEL SECURITIES AMERICAS SERVICES LLC', 'BLOOMBERG LP',
       'POINT72 ASSET MANAGEMENT LP', 'POINT72 LP', 'NDVR INC',
       'CITADEL AMERICAS SERVICES LLC', 'TWO SIGMA INVESTMENTS LP',
       'CITADEL ENTERPRISE AMERICAS SERVICES LLC', 'JUMP OPERATIONS LLC',
       'VIRTU FINANCIAL OPERATING LLC', 'SQUAREPOINT SERVICES US LLC',
       'CUBIST SYSTEMATIC STRATEGIES LLC', 'AFFIRM INC',
       'BALYASNY ASSET MANAGEMENT LP', 'MPG OPERATIONS LLC',
       'CITADEL SECURITIES AMERICAS LLC',
       'SCHONFELD STRATEGIC ADVISORS LLC', 'SOROS FUND MANAGEMENT LLC',
       'ARROWSTREET CAPITAL LIMITED PARTNERSHIP', 'OAK THISTLE LLC',
       'WELLS FARGO BANK NA', 'FEDERAL HOME LOAN MORTGAGE CORPORATION',
       'SCIENTECH RESEARCH LLC', 'LAURION CAPITAL MANAGEMENT LP',
       'CTC TRADING GROUP LLC', 'CENTIVA CAPITAL LP',
       'MANIFOLD CAPITAL PARTNERS LP',
       'PACIFIC INVESTMENT MANAGEMENT COMPANY LLC', 'J GOLDMAN & CO

In [28]:
df["FIRM"].value_counts(ascending=False)

CITADEL SECURITIES AMERICAS SERVICES LLC      39
CITADEL AMERICAS SERVICES LLC                 30
TWO SIGMA INVESTMENTS LP                      18
POINT72 LP                                    15
BALYASNY ASSET MANAGEMENT LP                  15
JUMP OPERATIONS LLC                           15
BLOOMBERG LP                                  12
MPG OPERATIONS LLC                             9
CTC TRADING GROUP LLC                          9
FEDERAL HOME LOAN MORTGAGE CORPORATION         9
BC USA GP LLC                                  9
CITIGROUP GLOBAL MARKETS INC                   6
SQUAREPOINT SERVICES US LLC                    6
SCHONFELD STRATEGIC ADVISORS LLC               6
PACIFIC INVESTMENT MANAGEMENT COMPANY LLC      6
SOROS FUND MANAGEMENT LLC                      6
ARROWSTREET CAPITAL LIMITED PARTNERSHIP        6
AFFIRM INC                                     6
TOWER RESEARCH CAPITAL LLC                     6
CITADEL ENTERPRISE AMERICAS SERVICES LLC       6
FIDELITY MANAGEMENT 